<a href="https://colab.research.google.com/github/fabiomoschella/Coursera_Capstone/blob/master/FM_Capstone_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Segmenting and Clustering Neighborhoods in Toronto¶
###PART 1:

In [0]:
import pandas as pd
import numpy  as np


I use only pandas method  for extract the table  and an old wikipedia link with correct table 


In [0]:
HtmlWeb='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'
df = pd.read_html(HtmlWeb, header = 0)[0]
df.rename(columns={'Postcode':'Postalcode'},inplace=True)



We process only Postcode with borough assigned,skipped everything else.


In [3]:
df = df[df.Borough != 'Not assigned']
df.head()


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [4]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. These multiple rows will be combined into one row with the neighborhoods separated with a comma.



In [5]:
df = df.groupby(['Borough', 'Postalcode'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()


,Borough,Postalcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


Totale records


In [6]:
df.shape

(103, 3)

###PART 2:



Use the Geocoder package to get the latitude and the longitude coordinates for all the neighborhoods in our dataframe. 



In [7]:
!pip install  geocoder 

import geocoder
import pandas as pd
import numpy as np

Read the file csv and rename the column 'Postal Code'  for the join subsequently.

In [8]:
csvfile='http://cocl.us/Geospatial_data'
dfGeo = pd.read_csv(csvfile)
dfGeo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
dfGeo

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Simple Join Dataframe between Borough and Geo information

In [9]:
sKey='Postalcode'
#BEGIN   
if sKey in df.columns: 
  df.set_index(sKey,inplace=True)
   
if sKey in dfGeo.columns: 
   dfGeo.set_index(sKey,inplace=True)
#END 

dfFinal= df.join(dfGeo,on='Postalcode',rsuffix='right_')

df.reset_index(inplace=True)
dfGeo.reset_index(inplace=True)
dfFinal.reset_index(inplace=True) 

dfFinal

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...
98,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
99,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
100,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
101,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262


### Part 3: Folium Map

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
1.   to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2.   to generate maps to visualize your neighborhoods and how they cluster together.



In [10]:
#!pip install folium
#!pip install  geopy 
from geopy.geocoders import Nominatim 

#with this library I take the latitude and longitude coordinates
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="FM_Capstone_Project_Notebook")
location = geolocator.geocode(address)
print('The geograpical coordinate of ',address,' are {}, {}.'.format(location.latitude, location.longitude))

The geograpical coordinate of  Toronto,ON  are 43.6534817, -79.3839347.


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dfFinal['Latitude'], dfFinal['Longitude'], dfFinal['Borough'], dfFinal['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
            
map_toronto